# Séries Temporais

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as srn
srn.set()

In [ ]:
#!pip install pmdarima

In [ ]:
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import auto_arima

## Exploração dos dados

In [ ]:
dataset = pd.read_csv('dados\AirPassengers.csv')
print(dataset.shape)
dataset.head()

In [ ]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m')
dataset = pd.read_csv('dados\Airpassengers.csv', parse_dates=['Month'], index_col='Month', date_parser=dateparse)

In [ ]:
dataset.info()  # month virou a coluna índice

In [ ]:
time_series = dataset['#Passengers']
time_series  # temos todas as explorações que o datetime permite

In [ ]:
print(time_series.index.max())  # mais novo
print(time_series.index.min())  # mais belho

In [ ]:
plt.plot(time_series);

In [ ]:
time_series_ano = time_series.resample('A').sum()
plt.plot(time_series_ano)  # por ano

In [ ]:
time_series_mes = time_series.groupby([lambda x: x.month]).sum()
plt.plot(time_series_mes);  # por mês

In [ ]:
time_series_datas = time_series['1960-01-01':'1960-12-01']
plt.plot(time_series_datas);  # por um periodo específico

## Decomposição da série temporal

In [ ]:
decomposicao = seasonal_decompose(time_series)

# pode ser dividido em trêscaracterística:
tendencia = decomposicao.trend
sazonal = decomposicao.seasonal
aleatorio = decomposicao.resid

In [ ]:
plt.plot(tendencia);
plt.title('Tendência da série temporal')
plt.show()

# aumenta com os anos

In [ ]:
plt.plot(sazonal);
plt.title('Sazonalidade da série temporal')
plt.show()

# pico nas férias (julho, agosto, setembro)

In [ ]:
plt.plot(aleatorio);
plt.title('Aleatoriedade da série temporal')
plt.show()

# sem padrão definido

## Previsões com ARIMA

In [ ]:
# Parâmetros P, Q e D
model = auto_arima(time_series, order=(2, 1, 2))

In [ ]:
previsoes = model.predict(n_periods=12)  
previsoes   # para 12 datas pra frente (um ano)

In [ ]:
len(time_series)

In [ ]:
train = time_series[:130]
test = time_series[130:]
train.shape, test.shape

In [ ]:
print(train.index.min(), train.index.max())
print(test.index.min(), test.index.max())

In [ ]:
model2 = auto_arima(train, suppress_warning=True)

In [ ]:
previsoes = pd.DataFrame(model2.predict(n_periods=14), index=test.index)
previsoes.columns = ['passengers_predictions']
previsoes.shape

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label='Training')
plt.plot(test, label='Testing')
plt.plot(previsoes, label='Predictions')
plt.legend()
plt.show();

As Previsoes obteram:
- um valor máximo menor do que os valores originais
- um valor mínimo menor do que os valores originais
- não teve a mesma forma

## Séries temporais com Facebook Prophet

In [ ]:
#!pip install fbprophet 

In [ ]:
from fbprophet import Prophet  # doesnt work for windows

In [ ]:
dataset = pd.read_csv('dados\page_wikipedia.csv')
print(dataset.shape)
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
dataset.hist();

In [ ]:
dataset = dataset[['date', 'views']].rename(columns={'date':'ds', 'views':'y'})
dataset = dataset.sort_values(by='ds')
dataset.head()

### Construindo o modelo

In [ ]:
model = Prophet()
model.fit(dataset)

In [ ]:
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)

In [ ]:
forecast.head()

In [ ]:
len(dataset), len(forecast)

In [ ]:
len(forecast) - len(dataset)

In [ ]:
forecast.tail(90)

### Gráfico das previsões